In [1]:
!pip install sentencepiece
!pip  install transformers
!pip install pytorch-ignite
!pip install datasets



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import argparse
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import random
import numpy as np
import torch.nn as nn
import torch
from transformers import AutoModel
import torch.nn.functional as F
from sklearn.metrics import f1_score, accuracy_score, classification_report
from transformers import AdamW, get_linear_schedule_with_warmup
# Any results you write to the current directory are saved as output.
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from transformers import BertTokenizer,BertModel
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset
from torch.nn.utils.rnn import pack_padded_sequence
from torch.optim import AdamW
from tqdm import tqdm
from argparse import ArgumentParser
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.engine.engine import Engine, State, Events
from ignite.handlers import EarlyStopping
from ignite.contrib.handlers import TensorboardLogger, ProgressBar
from ignite.utils import convert_tensor
from torch.optim.lr_scheduler import ExponentialLR
import warnings  
warnings.filterwarnings('ignore')
import gc
import copy
import time
import random
import string

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

# Utils
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold
from transformers import AutoTokenizer, AutoModel, AdamW
import random
import os
from urllib import request
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Preprocessing

In [3]:
class TrainDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.text = df['#2_content'].values
        self.label=df['#3_label'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        # summary = self.summary[index]
        inputs_text = self.tokenizer.encode_plus(
                                text,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        
                            
        target = self.label[index]
        
        text_ids = inputs_text['input_ids']
        text_mask = inputs_text['attention_mask']
        
       
        
        
        return {
            
            'text_ids': torch.tensor(text_ids, dtype=torch.long),
            'text_mask': torch.tensor(text_mask, dtype=torch.long),
            'target': torch.tensor(target, dtype=torch.float)
        }

In [4]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from torch.utils.data import Dataset, DataLoader
# import Dataset
# import os
# import numpy as np
# from emoji import UNICODE_EMOJI
# import TweetNormalizer
# import re
# import text_normalization


# dic = {
#       "egypt": 'المصرية',
# 	  "nile": 'المصرية',
# 	  "msa": "اللغة العربية الفصحى",
# 	  "magreb": "المغربية",
# 	  "gulf": "الخليجية",
# 	  "levant": "الشامية"
# }

# def is_emoji(s):
#     return s in UNICODE_EMOJI

# # add space near your emoji
# def add_space(text):
#     return ''.join(' ' + char if is_emoji(char) else char for char in text).strip()

# def preprocess(text, lang='ar'):
#     if lang == 'ar':
#         sent = add_space(text)
#         sent = re.sub(r'(?:@[\w_]+)', "user", sent)
#         sent = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', "url", sent)
#         sent = sent.replace('_', ' ')
#         sent = sent.replace('#', ' ')
#     else:
#         sent = add_space(text)
#         sent = re.sub(r'(?:@[\w_]+)', "@user", sent)
#         sent = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', "http", sent)
#         sent = sent.replace('_', ' ')
#         sent = sent.replace('#', ' ')

#     return sent

# def prepare_text(df, col='tweet'):
#     if col == 'tweet':
#         df['dialect'] = df['dialect'].map(dic)   
#     for i in range(df.shape[0]):
#         df.loc[i, col] = df.loc[i, 'dialect'] + ' [SEP] ' + df.loc[i, col]


#     return df

# def augment_data(df_train,text_col,label_col):
#     df_aug = pd.DataFrame(columns=[text_col, label_col)
#     dic_dup = {1: 3,
#                0: 1
#                }
#     for i in range(df_train.shape[0]):
#         current = df_train.iloc[i]
#         text = current[text_col]
#         label_cat = current[label_col]

#         aug_ratio = dic_dup[label_cat]
#         for k in range(aug_ratio):
#             tokens = text.split(' ')
#             l = len(tokens)
#             n = int(0.1 * l)
#             indices = np.random.choice(l, n, replace=False)
#             for j in range(len(indices)):
#                 tokens[indices[j]] = '[MASK]'
#             new_text = ' '.join(tokens)
#             entry = {text_col: new_text, label_col: label_cat}
#             df_aug = df_aug.append(entry, ignore_index=True)
#     df_aug.drop_duplicates(subset=[text_col], keep='first', inplace=True)
#     df = pd.concat([df_train,df_aug])
#     return df


In [5]:
# '''
# Created by: Mohamed Salem Elhady  
# Email: mohamed.elaraby@alumni.ubc.ca
# Text Normalization: V1 
# '''
# import sys
# import re
# import emojis
# from emoji import UNICODE_EMOJI
# #sys.setdefaultencoding('utf-8')
# ##########################Clean Text Data #######################################
# ########################Global Variable Declaration##############################
# list_seeds = ['سبحان الله', 'الله أكبر', 'اللهم', 'بسم الله', 'يا رب', 'العضيم', 'سبحان', 'يارب', 'قران', 'quran',
#               'حديث', 'hadith', 'صلاه_الفجر', '﴾', 'ﷺ', 'صحيح البخاري', 'صحيح مسلم', 'يآرب', 'سورة']
# MaxWordPerTweet=7
# #################################################################################
# def is_emoji(s):
#     return s in UNICODE_EMOJI

# # add space near your emoji
# def add_space(text):
#     return ''.join(' ' + char if is_emoji(char) else char for char in text).strip()

# def clean(sent):
#     """clean data from any English char, emoticons, underscore, and repeated > 2
#     str -> str"""
#     p1 = re.compile('\W')
#     p2 = re.compile('\s+')
#     sent = re.sub(r"http\S+", "", sent)
#     sent = ReplaceThreeOrMore(sent)
#     sent = remove_unicode_diac(sent)
#     sent = sent.replace('_', ' ')
#     sent = re.sub(r'[A-Za-z0-9]', r'', sent)
#     sent = re.sub(p1, ' ', sent)
#     sent = re.sub(p2, ' ', sent)
#     return sent

# def tokenize_emojis(tweet):
#     return list(emojis.get(tweet))

# def replace_emoji(sent):
#     emoji_pattern = re.compile("["
#                                u"\U0001F600-\U0001F64F"  # emoticons
#                                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#                                u"\U0001F680-\U0001F6FF"  # transport & map symbols
#                                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                                "]+", flags=re.UNICODE)
#     return emoji_pattern.sub(r'[MASK]', sent)

# def preprocess(tweet):
#     tweet = add_space(tweet)
#     emos = tokenize_emojis(tweet)
#     sent = remove_unicode_diac(tweet)
#     sent = re.sub(r'(?:@[\w_]+)', "user", sent)
#     sent = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', "url", sent)
#     sent = sent.replace('_', ' ')
#     sent = sent.replace('#', ' ')
#     if len(emos) > 0:
#         sent = sent + ' [SEP] '  + ' '.join(emos)
#     #    #sent = sent + ' [SEP] ' + clean_unicode(tweet) + ' [SEP] ' + ' '.join(emos)

#     #else:
#     #    sent = sent + ' [SEP] ' + clean_unicode(tweet)
#     return sent

# def preprocess_last(tweet, k=0):
#     emos = tokenize_emojis(tweet)
#     sent = remove_unicode_diac(tweet)
#     sent = re.sub(r'(?:@[\w_]+)', "", sent)
#     sent = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', "", sent)
#     sent = sent.replace('_', ' ')
#     sent = sent.replace('#', ' ')
#     if k == 0:
#         sent = sent
#     elif k ==1 :
#         sent = sent + ' [SEP] ' + ' '.join(emos)
#     elif k==2 :
#         sent = sent + ' [SEP] ' + clean_unicode(tweet) + ' [SEP] ' + ' '.join(emos)
#     elif k == 3:
#         sent = sent + ' [SEP] ' + clean_unicode(tweet)
#     else:
#         sent = replace_emoji(sent)
#         sent = sent + ' [SEP] ' + clean_unicode(tweet) + ' [SEP] ' + ' '.join(emos)

#     return sent


# def normalize(sent):
#     """clean data from any English char, emoticons, underscore, and repeated > 2
#     str -> str"""
#     sent = re.sub(r'(?:@[\w_]+)', "user", sent)
#     sent = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', "url", sent)
#     #sent = re.sub(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", "hashtag", sent)
#     sent = ReplaceThreeOrMore(sent)
#     sent = remove_unicode_diac(sent)
#     sent = sent.replace('_', ' ')
#     return sent

# def ReplaceThreeOrMore(s):
#     # pattern to look for three or more repetitions of any character, including
#     # newlines.
#     pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
#     return pattern.sub(r"\1\1", s)
# def norm_alif(text):
#     text = text.replace(u"\u0625", u"\u0627")  # HAMZA below, with LETTER ALEF
#     #text = text.replace(u"\u0621", u"\u0627")  # HAMZA, with LETTER ALEF
#     text = text.replace(u"\u0622", u"\u0627")  # ALEF WITH MADDA ABOVE, with LETTER ALEF
#     text = text.replace(u"\u0623", u"\u0627")  # ALEF WITH HAMZA ABOVE, with LETTER ALEF
#     return text
# def remove_unicode_diac(text):
#     """Takes Arabic in utf-8 and returns same text without diac"""
#     # Replace diacritics with nothing
#     text = text.replace(u"\u064B", "")  # fatHatayn
#     text = text.replace(u"\u064C", "")  # Dammatayn
#     text = text.replace(u"\u064D", "")  # kasratayn
#     text = text.replace(u"\u064E", "")  # fatHa
#     text = text.replace(u"\u064F", "")  # Damma
#     text = text.replace(u"\u0650", "")  # kasra
#     text = text.replace(u"\u0651", "")  # shaddah
#     text = text.replace(u"\u0652", "")  # sukuun
#     text = text.replace(u"\u0670", "`")  # dagger 'alif
#     return text
# def norm_taa(text):
#     text=text.replace(u"\u0629", u"\u0647") # taa' marbuuTa, with haa'
#     #text=text.replace(u"\u064A", u"\u0649") # yaa' with 'alif maqSuura
#     return text
# def norm_yaa(text):
#     if len(text)!=0:
#         if text[-1] == u"\u064A":
#             text = text[:-1] + text[-1].replace(u"\u064A", u"\u0649")  # yaa' with 'alif maqSuura
#     return text

# def NormForWord2Vec(text):
#     text=norm_taa(text)
#     text=norm_yaa(text)
#     text=norm_alif(text)
#     return text

# def remove_nonunicode2(Tweet):
#     ## defining set of unicode ##
#     #u""
#     #Tweet=Tweet.decode("utf-8")
#     UniLex={ ## This is list of all arabic unicode characters in addition to space (to separate words)
#             u"\u0622",
#             u"\u0626",
#             u"\u0628",
#             u"\u062a",
#             u"\u062c",
#             u"\u06af",
#             u"\u062e",
#             u"\u0630",
#             u"\u0632",
#             u"\u0634",
#             u"\u0636",
#             u"\u0638",
#             u"\u063a",
#             u"\u0640",
#             u"\u0642",
#             u"\u0644",
#             u"\u0646",
#             u"\u0648",
#             u"\u064a",
#             u"\u0670",
#             u"\u067e",
#             u"\u0686",
#             u"\u0621",
#             u"\u0623",
#             u"\u0625",
#             u"\u06a4",
#             u"\u0627",
#             u"\u0629",
#             u"\u062b",
#             u"\u062d",
#             u"\u062f",
#             u"\u0631",
#             u"\u0633",
#             u"\u0635",
#             u"\u0637",
#             u"\u0639",
#             u"\u0641",
#             u"\u0643",
#             u"\u0645",
#             u"\u0647",
#             u"\u0649",
#             u"\u0671",
#             ' ',
#             '\n'
#           }
#     fin_tweet=""
#     for c in Tweet:
#         if c in UniLex:
#            fin_tweet=fin_tweet+c
#     return fin_tweet

# ###### Heuristics Calculations ######
# def diac_counter(text):
#     #text=text.decode("utf-8")
#     diac = [u"\u064B",u"\u064C", u"\u064D", u"\u064E", u"\u064F", u"\u0650", u"\u0651", u"\u0652", u"\u0670"]
#     diac_count=0
#     for d in diac:
#         diac_count+=text.count(d)
# #         if d in text:
# #             print(d)
# #             diac_count+=1
#     return diac_count
# def check_seed(list_seeds, text):
#     ""
#     for word in list_seeds:
#         text = text.lower()
#         if word.decode("utf-8") in text:
#             return True
#     return False
# def EnglishCount(text):
#     printable = ['e', 'a', 'o', 't', 'i']
#     count = 0
#     for ch in printable:
#         count += text.count(ch.lower())
#     return count
# ########################################



# def eliminate_single_char_words(Tweet):
#     parts = Tweet.split(" ")
#     cleaned_line_parts = []
#     for P in parts:
#         if len(P) != 1:
#             cleaned_line_parts.append(P)
#     cleaned_line = ' '.join(cleaned_line_parts)
#     return cleaned_line
# def clean_unicode(Tweet):
#     tweet=normalize(Tweet.strip("\n"))
#     if len(tweet) !=0:
#         sentence = []
#         for word in tweet.split(" "):
#             word = remove_unicode_diac(word)
#             word = norm_alif(word)
#             word = norm_taa(word)
#             word = norm_yaa(word)
#             word = normalize(word)
#             sentence.append(word)
#         tweet = ' '.join(sentence)
#         tweet =remove_nonunicode2(tweet)
#         tweet =eliminate_single_char_words(tweet)
#     return tweet

# def clean_unicode2(Tweet):
#     KeepUniOnly(Tweet)
#     tweet=normalize(Tweet.strip("\n"))
#     if len(tweet) !=0:
#         sentence = []
#         for word in tweet.split(" "):
#             word = remove_unicode_diac(word)
#             word = normalize(word)
#             sentence.append(word)
#         tweet = ' '.join(sentence)
#         tweet =remove_nonunicode2(tweet)
#         tweet =eliminate_single_char_words(tweet)
#     return tweet

# def NormCorpusFinal(Tweet):
#     tweet=KeepUniOnly(Tweet)
#     tweet=NormForWord2Vec(tweet)
#     return tweet

# def KeepUniOnly(Tweet):## this one is without normalization
#     tweet=Tweet.replace("# "," ")
#     tweet=tweet.replace("#"," ")
#     tweet=tweet.replace("_"," ")
#     tweet=tweet.replace(u"\u0657"," ")
#     tweet=tweet.replace("\n"," ")
#     tweet=remove_nonunicode2(tweet)
#     tweet=eliminate_single_char_words(tweet)
#     tweet=ReplaceThreeOrMore(tweet)
#     return tweet

# def get_charset(rawtext):
#     chars = sorted(list(set(rawtext)))
#     return chars

# def DialectChecker(text):
#     ##Based on Hueristics done by Hassan
#     if (diac_counter(text)>5 or check_seed(list_seeds,text) or EnglishCount(text)>4 or "<URL>"  in text
#         or text.count('#') >2 or '"'  in text or text.count('@') or "\"RT" in text or len(text.split(" ")) <7):
#         return False
#     else:
#         return True

# ###############################################################
# '''
# Fread=open("Egypt_portion.txt",'r')
# Fwriter=open("Egypt_portion_norm.txt",'w')
# for line in Fread:
#     cleaned_line=clean_unicode_for_w2v(line)
#     Fwriter.write(str(cleaned_line))
# Fwriter.close()
# '''

# Losses

In [6]:
class F1_Loss(nn.Module):
    '''Calculate F1 score. Can work with gpu tensors
    
    The original implmentation is written by Michal Haltuf on Kaggle.
    
    Returns
    -------
    torch.Tensor
        `ndim` == 1. epsilon <= val <= 1
    
    Reference
    ---------
    - https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric
    - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
    - https://discuss.pytorch.org/t/calculating-precision-recall-and-f1-score-in-case-of-multi-label-classification/28265/6
    - http://www.ryanzhang.info/python/writing-your-own-loss-function-module-for-pytorch/
    '''
    def __init__(self, epsilon=1e-7):
        super().__init__()
        self.epsilon = epsilon
        
    def forward(self, y_pred, y_true,):
        # assert y_pred.ndim == 2
        # assert y_true.ndim == 1
        # print(y_pred.shape)
        # print(y_true.shape)
        # y_pred[y_pred<0.5]=0
        # y_pred[y_pred>=0.5]=0


        
        y_true_one_hot = F.one_hot(y_true.to(torch.int64), 18).to(torch.float32)
        # y_pred_one_hot = F.one_hot(y_pred.to(torch.int64), 2).to(torch.float32)
        
        tp = (y_true_one_hot * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true_one_hot) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true_one_hot) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true_one_hot * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        f1 = 2* (precision*recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min=self.epsilon, max=1-self.epsilon)
        f1=f1.detach()
        # print(f1.shape)
        # y_pred=y_pred.reshape((y_pred.shape[0], 1))
        # y_true=y_true.reshape((y_true.shape[0], 1))

        # p1=y_true*(math.log(sigmoid(y_pred)))*(1-f1)[1]
        # p0=(1-y_true)*math.log(1-sigmoid(y_pred))*(1-f1)[0]


        # y_true_one_hot = F.one_hot(y_true.to(torch.int64), 2)
        # print(y_pred)
        # print(y_true_one_hot)
        CE =torch.nn.CrossEntropyLoss(weight=( 1 - f1))(y_pred, y_true_one_hot)
        # loss = ( 1 - f1)  * CE
        return  CE.mean()

In [7]:
class Recall_Loss(nn.Module):
    '''Calculate Recall score. Can work with gpu tensors
    
    The original implmentation is written by Michal Haltuf on Kaggle.
    
    Returns
    -------
    torch.Tensor
        `ndim` == 1. epsilon <= val <= 1
    
    Reference
    ---------
    - https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric
    - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
    - https://discuss.pytorch.org/t/calculating-precision-recall-and-f1-score-in-case-of-multi-label-classification/28265/6
    - http://www.ryanzhang.info/python/writing-your-own-loss-function-module-for-pytorch/
    '''
    def __init__(self, epsilon=1e-7):
        super().__init__()
        self.epsilon = epsilon
        
    def forward(self, y_pred, y_true,):
        # assert y_pred.ndim == 2
        # assert y_true.ndim == 1
        # print(y_pred.shape)
        # print(y_true.shape)
        # y_pred[y_pred<0.5]=0
        # y_pred[y_pred>=0.5]=0


        
        y_true_one_hot = F.one_hot(y_true.to(torch.int64), 18).to(torch.float32)
        # y_pred_one_hot = F.one_hot(y_pred.to(torch.int64), 2).to(torch.float32)
        
        tp = (y_true_one_hot * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true_one_hot) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true_one_hot) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true_one_hot * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        # f1 = 2* (precision*recall) / (precision + recall + self.epsilon)
        # f1 = f1.clamp(min=self.epsilon, max=1-self.epsilon)
        # f1=f1.detach()
        # print(f1.shape)
        # y_pred=y_pred.reshape((y_pred.shape[0], 1))
        # y_true=y_true.reshape((y_true.shape[0], 1))

        # p1=y_true*(math.log(sigmoid(y_pred)))*(1-f1)[1]
        # p0=(1-y_true)*math.log(1-sigmoid(y_pred))*(1-f1)[0]


        # y_true_one_hot = F.one_hot(y_true.to(torch.int64), 2)
        # print(y_pred)
        # print(y_true_one_hot)
        recall=recall.detach()
        CE =torch.nn.CrossEntropyLoss(weight=( 1 - recall))(y_pred, y_true_one_hot)
        # loss = ( 1 - f1)  * CE
        return  CE.mean()

In [8]:
## Based on https://github.com/AbdelkaderMH/iSarcasmEval/blob/8f28f24ebfb641415a604329ed859506ae687148/focal_loss.py
class BinaryFocalLoss(nn.Module):
    """
    This is a implementation of Focal Loss with smooth label cross entropy supported which is proposed in
    'Focal Loss for Dense Object Detection. (https://arxiv.org/abs/1708.02002)'
        Focal_Loss= -1*alpha*(1-pt)*log(pt)
    :param alpha: (tensor) 3D or 4D the scalar factor for this criterion
    :param gamma: (float,double) gamma > 0 reduces the relative loss for well-classified examples (p>0.5) putting more
                    focus on hard misclassified example
    :param reduction: `none`|`mean`|`sum`
    :param **kwargs
        balance_index: (int) balance class index, should be specific when alpha is float
    """

    def __init__(self, alpha=3, gamma=2, ignore_index=None, reduction='mean', **kwargs):
        super(BinaryFocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.smooth = 1e-6  # set '1e-4' when train with FP16
        self.ignore_index = ignore_index
        self.reduction = reduction

        assert self.reduction in ['none', 'mean', 'sum']

        # if self.alpha is None:
        #     self.alpha = torch.ones(2)
        # elif isinstance(self.alpha, (list, np.ndarray)):
        #     self.alpha = np.asarray(self.alpha)
        #     self.alpha = np.reshape(self.alpha, (2))
        #     assert self.alpha.shape[0] == 2, \
        #         'the `alpha` shape is not match the number of class'
        # elif isinstance(self.alpha, (float, int)):
        #     self.alpha = np.asarray([self.alpha, 1.0 - self.alpha], dtype=np.float).view(2)

        # else:
        #     raise TypeError('{} not supported'.format(type(self.alpha)))

    def forward(self, output, target):
        prob = torch.sigmoid(output)
        prob = torch.clamp(prob, self.smooth, 1.0 - self.smooth)

        valid_mask = None
        if self.ignore_index is not None:
            valid_mask = (target != self.ignore_index).float()

        pos_mask = (target == 1).float()
        neg_mask = (target == 0).float()
        if valid_mask is not None:
            pos_mask = pos_mask * valid_mask
            neg_mask = neg_mask * valid_mask

        pos_weight = (pos_mask * torch.pow(1 - prob, self.gamma)).detach()
        pos_loss = -pos_weight * torch.log(prob)  # / (torch.sum(pos_weight) + 1e-4)

        neg_weight = (neg_mask * torch.pow(prob, self.gamma)).detach()
        neg_loss = -self.alpha * neg_weight * F.logsigmoid(-output)  # / (torch.sum(neg_weight) + 1e-4)
        loss = pos_loss + neg_loss
        loss = loss.mean()
        return loss


class FocalLoss_Ori(nn.Module):
    """
    This is a implementation of Focal Loss with smooth label cross entropy supported which is proposed in
    'Focal Loss for Dense Object Detection. (https://arxiv.org/abs/1708.02002)'
    Focal_Loss= -1*alpha*((1-pt)**gamma)*log(pt)
    Args:
        num_class: number of classes
        alpha: class balance factor
        gamma:
        ignore_index:
        reduction:
    """

    def __init__(self, num_class, alpha=None, gamma=2, ignore_index=None, reduction='mean'):
        super(FocalLoss_Ori, self).__init__()
        self.num_class = num_class
        self.gamma = gamma
        self.reduction = reduction
        self.smooth = 1e-4
        self.ignore_index = ignore_index
        self.alpha = alpha
        if alpha is None:
            self.alpha = torch.ones(num_class, )
        elif isinstance(alpha, (int, float)):
            self.alpha = torch.as_tensor([alpha] * num_class)
        elif isinstance(alpha, (list, np.ndarray)):
            self.alpha = torch.as_tensor(alpha)
        if self.alpha.shape[0] != num_class:
            raise RuntimeError('the length not equal to number of class')

        # if isinstance(self.alpha, (list, tuple, np.ndarray)):
        #     assert len(self.alpha) == self.num_class
        #     self.alpha = torch.Tensor(list(self.alpha))
        # elif isinstance(self.alpha, (float, int)):
        #     assert 0 < self.alpha < 1.0, 'alpha should be in `(0,1)`)'
        #     assert balance_index > -1
        #     alpha = torch.ones((self.num_class))
        #     alpha *= 1 - self.alpha
        #     alpha[balance_index] = self.alpha
        #     self.alpha = alpha
        # elif isinstance(self.alpha, torch.Tensor):
        #     self.alpha = self.alpha
        # else:
        #     raise TypeError('Not support alpha type, expect `int|float|list|tuple|torch.Tensor`')

    def forward(self, logit, target):
        # assert isinstance(self.alpha,torch.Tensor)\
        N, C = logit.shape[:2]
        alpha = self.alpha.to(logit.device)
        prob = F.softmax(logit, dim=1)
        if prob.dim() > 2:
            # N,C,d1,d2 -> N,C,m (m=d1*d2*...)
            prob = prob.view(N, C, -1)
            prob = prob.transpose(1, 2).contiguous()  # [N,C,d1*d2..] -> [N,d1*d2..,C]
            prob = prob.view(-1, prob.size(-1))  # [N,d1*d2..,C]-> [N*d1*d2..,C]
        ori_shp = target.shape
        target = target.view(-1, 1)  # [N,d1,d2,...]->[N*d1*d2*...,1]
        valid_mask = None
        if self.ignore_index is not None:
            valid_mask = target != self.ignore_index
            target = target * valid_mask

        # ----------memory saving way--------
        prob = prob.gather(1, target).view(-1) + self.smooth  # avoid nan
        logpt = torch.log(prob)
        # alpha_class = alpha.gather(0, target.view(-1))
        alpha_class = alpha[target.squeeze().long()]
        class_weight = -alpha_class * torch.pow(torch.sub(1.0, prob), self.gamma)
        loss = class_weight * logpt
        if valid_mask is not None:
            loss = loss * valid_mask.squeeze()

        if self.reduction == 'mean':
            loss = loss.mean()
            if valid_mask is not None:
                loss = loss.sum() / valid_mask.sum()
        elif self.reduction == 'none':
            loss = loss.view(ori_shp)
        return loss



def weighted_binary_cross_entropy(input, targets, pos_weight, weight=None, size_average=True, reduce=True):
    """
    Args:
        sigmoid_x: predicted probability of size [N,C], N sample and C Class. Eg. Must be in range of [0,1], i.e. Output from Sigmoid.
        targets: true value, one-hot-like vector of size [N,C]
        pos_weight: Weight for postive sample
    """
    sigmoid_x = torch.sigmoid(input)
    if not (targets.size() == sigmoid_x.size()):
        raise ValueError("Target size ({}) must be the same as input size ({})".format(targets.size(), sigmoid_x.size()))

    loss = -pos_weight* targets * sigmoid_x.log() - (1-targets)*(1-sigmoid_x).log()

    if weight is not None:
        loss = loss * weight

    if not reduce:
        return loss
    elif size_average:
        return loss.mean()
    else:
        return loss.sum()

class WeightedBCELoss(nn.Module):
    def __init__(self, pos_weight= 1, weight=None, PosWeightIsDynamic= True, WeightIsDynamic= False, size_average=True, reduce=True):
        """
        Args:
            pos_weight = Weight for postive samples. Size [1,C]
            weight = Weight for Each class. Size [1,C]
            PosWeightIsDynamic: If True, the pos_weight is computed on each batch. If pos_weight is None, then it remains None.
            WeightIsDynamic: If True, the weight is computed on each batch. If weight is None, then it remains None.
        """
        super().__init__()

        #self.register_buffer('weight', weight)
        #self.register_buffer('pos_weight', pos_weight)
        self.size_average = size_average
        self.reduce = reduce
        self.PosWeightIsDynamic = PosWeightIsDynamic

    def forward(self, input, target):
        # pos_weight = Variable(self.pos_weight) if not isinstance(self.pos_weight, Variable) else self.pos_weight
        if self.PosWeightIsDynamic:
            positive_counts = target.sum(dim=0)
            nBatch = len(target)
            self.pos_weight = (nBatch - positive_counts)/(positive_counts +1e-5)


        return weighted_binary_cross_entropy(input, target,
                                                self.pos_weight,
                                                weight=None,
                                                size_average=self.size_average,
                                                reduce=self.reduce)


class WeightedCELoss(nn.Module):
    def __init__(self, size_average=True, reduce=True):
        """
        Args:
            pos_weight = Weight for postive samples. Size [1,C]
            weight = Weight for Each class. Size [1,C]
            PosWeightIsDynamic: If True, the pos_weight is computed on each batch. If pos_weight is None, then it remains None.
            WeightIsDynamic: If True, the weight is computed on each batch. If weight is None, then it remains None.
        """
        super().__init__()

        self.size_average = size_average
        self.reduce = reduce

    def forward(self, input, target):
        positive_counts = target.sum(dim=0)
        nBatch = len(target)
        pos_weight = (nBatch - positive_counts)/(positive_counts +1e-5)
        neg_count = nBatch - positive_counts
        neg_weight = (nBatch - neg_count)/(neg_count +1e-5)

        weight = torch.tensor([neg_weight, pos_weight], device=target.device)
  


        return F.cross_entropy(input, target, weight=weight)



class FLMultiLoss(nn.Module):
    def __init__(self, gamma= 2):
        """
        Args:
            pos_weight = Weight for postive samples. Size [1,C]
            weight = Weight for Each class. Size [1,C]
            PosWeightIsDynamic: If True, the pos_weight is computed on each batch. If pos_weight is None, then it remains None.
            WeightIsDynamic: If True, the weight is computed on each batch. If weight is None, then it remains None.
        """
        super().__init__()

        self.gamma = gamma
    def forward(self, input, target):


        return focal_binary_cross_entropy(input, target, gamma=2)

def EntropyLoss(input_):
    mask = input_.ge(0.000001)
    mask_out = torch.masked_select(input_, mask)
    entropy = -(torch.sum(mask_out * torch.log(mask_out)))
    return entropy / float(input_.size(0))

def focal_binary_cross_entropy(logits, targets, gamma=2):
    num_label = targets.shape[1]
    l = logits.reshape(-1)
    t = targets.reshape(-1)
    p = torch.sigmoid(l)
    p = torch.where(t >= 0.5, p, 1-p)
    logp = - torch.log(torch.clamp(p, 1e-4, 1-1e-4))
    loss = logp*((1-p)**gamma)
    loss = num_label*loss.mean()
    return loss

In [9]:
from typing import Optional, Sequence

import torch
from torch import Tensor
from torch import nn
from torch.nn import functional as F


class FocalLoss(nn.Module):
    """ Focal Loss, as described in https://arxiv.org/abs/1708.02002.
    It is essentially an enhancement to cross entropy loss and is
    useful for classification tasks when there is a large class imbalance.
    x is expected to contain raw, unnormalized scores for each class.
    y is expected to contain class labels.
    Shape:
        - x: (batch_size, C) or (batch_size, C, d1, d2, ..., dK), K > 0.
        - y: (batch_size,) or (batch_size, d1, d2, ..., dK), K > 0.
    """

    def __init__(self,
                 alpha: Optional[Tensor] = None,
                 gamma: float = 0.,
                 reduction: str = 'mean',
                 ignore_index: int = -100):
        """Constructor.
        Args:
            alpha (Tensor, optional): Weights for each class. Defaults to None.
            gamma (float, optional): A constant, as described in the paper.
                Defaults to 0.
            reduction (str, optional): 'mean', 'sum' or 'none'.
                Defaults to 'mean'.
            ignore_index (int, optional): class label to ignore.
                Defaults to -100.
        """
        if reduction not in ('mean', 'sum', 'none'):
            raise ValueError(
                'Reduction must be one of: "mean", "sum", "none".')

        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ignore_index = ignore_index
        self.reduction = reduction

        self.nll_loss = nn.NLLLoss(
            weight=alpha, reduction='none', ignore_index=ignore_index)

    def __repr__(self):
        arg_keys = ['alpha', 'gamma', 'ignore_index', 'reduction']
        arg_vals = [self.__dict__[k] for k in arg_keys]
        arg_strs = [f'{k}={v}' for k, v in zip(arg_keys, arg_vals)]
        arg_str = ', '.join(arg_strs)
        return f'{type(self).__name__}({arg_str})'

    def forward(self, x: Tensor, y: Tensor) -> Tensor:
        if x.ndim > 2:
            # (N, C, d1, d2, ..., dK) --> (N * d1 * ... * dK, C)
            c = x.shape[1]
            x = x.permute(0, *range(2, x.ndim), 1).reshape(-1, c)
            # (N, d1, d2, ..., dK) --> (N * d1 * ... * dK,)
            y = y.view(-1)

        unignored_mask = y != self.ignore_index
        y = y[unignored_mask]
        if len(y) == 0:
            return torch.tensor(0.)
        x = x[unignored_mask]

        # compute weighted cross entropy term: -alpha * log(pt)
        # (alpha is already part of self.nll_loss)
        log_p = F.log_softmax(x, dim=-1)
        ce = self.nll_loss(log_p, y)

        # get true class column from each row
        all_rows = torch.arange(len(x))
        log_pt = log_p[all_rows, y]

        # compute focal term: (1 - pt)^gamma
        pt = log_pt.exp()
        focal_term = (1 - pt)**self.gamma

        # the full loss: -alpha * ((1 - pt)^gamma) * log(pt)
        loss = focal_term * ce

        if self.reduction == 'mean':
            loss = loss.mean()
        elif self.reduction == 'sum':
            loss = loss.sum()

        return loss


def focal_loss(alpha: Optional[Sequence] = None,
               gamma: float = 0.,
               reduction: str = 'mean',
               ignore_index: int = -100,
               device='cuda',
               dtype=torch.float32) -> FocalLoss:
    """Factory function for FocalLoss.
    Args:
        alpha (Sequence, optional): Weights for each class. Will be converted
            to a Tensor if not None. Defaults to None.
        gamma (float, optional): A constant, as described in the paper.
            Defaults to 0.
        reduction (str, optional): 'mean', 'sum' or 'none'.
            Defaults to 'mean'.
        ignore_index (int, optional): class label to ignore.
            Defaults to -100.
        device (str, optional): Device to move alpha to. Defaults to 'cpu'.
        dtype (torch.dtype, optional): dtype to cast alpha to.
            Defaults to torch.float32.
    Returns:
        A FocalLoss object
    """
    if alpha is not None:
        if not isinstance(alpha, Tensor):
            alpha = torch.tensor(alpha)
        alpha = alpha.to(device=device, dtype=dtype)

    fl = FocalLoss(
        alpha=alpha,
        gamma=gamma,
        reduction=reduction,
        ignore_index=ignore_index)
    return fl

# Models

In [10]:
def freeze(module):
    """
    Freezes module's parameters.
    """
    for parameter in module.parameters():
        parameter.requires_grad = False

def get_freezed_parameters(module):
    """
    Returns names of freezed parameters of the given module.
    """
    freezed_parameters = []
    for name, parameter in module.named_parameters():
        if not parameter.requires_grad:
            freezed_parameters.append(name)

In [35]:
import torch.nn as nn
import torch
from transformers import AutoModel
import torch.nn.functional as F

def init_weights(m):
    classname = m.__class__.__name__
    if classname.find('Conv2d') != -1 or classname.find('ConvTranspose2d') != -1:
        nn.init.kaiming_uniform_(m.weight)
        nn.init.zeros_(m.bias)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight, 1.0, 0.02)
        nn.init.zeros_(m.bias)
    elif classname.find('Linear') != -1:
        nn.init.xavier_normal_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)


class AttentionWithContext(nn.Module):
    def __init__(self, hidden_dim):
        super(AttentionWithContext, self).__init__()

        self.attn = nn.Linear(hidden_dim, hidden_dim)
        self.contx = nn.Linear(hidden_dim, 1, bias=False)
        #self.apply(init_weights)
    def forward(self, inp):
        u = torch.tanh_(self.attn(inp))
        a = F.softmax(self.contx(u), dim=1)
        s = (a * inp).sum(1)
        return s


class TransformerLayer(nn.Module):
    def __init__(self,
                 pretrained_path='aubmindlab/bert-base-arabert'):
        super(TransformerLayer, self).__init__()

        
        self.transformer = AutoModel.from_pretrained(pretrained_path, output_hidden_states=True)


    def forward(self, input_ids=None, attention_mask=None):
        outputs = self.transformer(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        #(output_last_layer, pooled_cls, (output_layers))
        #output[0] (8, seqlen=64, 768) cls [8, 768] ( 12 (8, seqlen=64, 768))

        return outputs

    def output_num(self):
        return self.transformer.config.hidden_size

class ATTClassifier(nn.Module):
    def __init__(self, in_feature, class_num=1, dropout_prob=0.2):
        super(ATTClassifier, self).__init__()
        self.attention = AttentionWithContext(in_feature)

        self.Classifier = nn.Sequential(
            nn.Linear(2 * in_feature, 512),
            nn.Dropout(dropout_prob),
            nn.ReLU(),
            nn.Linear(512, class_num)
        )

        self.apply(init_weights)

    def forward(self, x):
        att = self.attention(x[0]) #(X[0] (bs, seqlenght, embedD) att = \sum_i alpha_i x[0][i]

        xx = torch.cat([att, x[1]], 1)

        out = self.Classifier(xx)
        return out

class NADIModel(nn.Module):
  def __init__(self, pretrained_path='aubmindlab/bert-base-arabert',in_feature=768, class_num=18, dropout_prob=0.2):
        super(NADIModel, self).__init__()
        self.base_model = AutoModel.from_pretrained(pretrained_path, output_hidden_states=True)
        freeze(self.base_model.embeddings)
        freeze(self.base_model.encoder.layer[:2])
        self.classifier=ATTClassifier(self.base_model.config.hidden_size, class_num=class_num).to('cuda')
  def forward(self, ids,mask):
    output=self.base_model(ids,mask)
    output=self.classifier(output)
    # output=torch.softmax(output, dim=1)
    return output
 

# Train, Validation

In [36]:
class ImbalancedDatasetSampler(torch.utils.data.sampler.Sampler):
    """
    Samples elements randomly from a given list of indices for imbalanced dataset
    Arguments:
        indices (list, optional): a list of indices
        num_samples (int, optional): number of samples to draw
    """

    def __init__(self, dataset, indices=None, num_samples=None):
        # if indices is not provided,
        # all elements in the dataset will be considered
        self.indices = list(range(len(dataset['#3_label']))) \
            if indices is None else indices

        # if num_samples is not provided,
        # draw `len(indices)` samples in each iteration
        self.num_samples = len(self.indices) \
            if num_samples is None else num_samples

        # distribution of classes in the dataset
        label_to_count = {}
        for idx in self.indices:
            label = self._get_label(dataset, idx)
            if label in label_to_count:
                label_to_count[label] += 1
            else:
                label_to_count[label] = 1

        # weight for each sample
        weights = [1.0 / label_to_count[self._get_label(dataset, idx)] for idx in self.indices]
        self.weights = torch.DoubleTensor(weights)

    def _get_label(self, dataset, id_):
        return dataset['#3_label'][id_]

    def __iter__(self):
        return (self.indices[i] for i in torch.multinomial(self.weights, self.num_samples, replacement=True))

    def __len__(self):
        return self.num_samples

In [37]:
def criterion(outputs1,  targets):

    criterion = FocalLoss()
    loss = criterion(outputs1, targets)
    return loss

In [38]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        
        text_ids = data['text_ids'].to(device, dtype = torch.long)
        text_mask = data['text_mask'].to(device, dtype = torch.long)
        targets = data['target'].to(device, dtype=torch.long)
        
        batch_size = text_ids.size(0)
        # print(targets)

        outputs = model(text_ids, text_mask)
        # print(outputs.shape)
        
        # print(outputs.shape)

        
        loss = criterion(outputs, targets)
        loss = loss / CONFIG['n_accumulate']
        loss.backward()
    
        if (step + 1) % CONFIG['n_accumulate'] == 0:
            optimizer.step()

            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
    gc.collect()
    
    return epoch_loss

In [39]:
from sklearn.metrics import jaccard_score,f1_score,accuracy_score,recall_score,precision_score,classification_report
def print_statistics(y, y_pred):
    accuracy = accuracy_score(y, y_pred)
    precision =precision_score(y, y_pred, average='weighted')
    recall = recall_score(y, y_pred, average='weighted')
    f_score = f1_score(y, y_pred, average='weighted')
    print('Accuracy: %.3f\nPrecision: %.3f\nRecall: %.3f\nF_score: %.3f\n'
          % (accuracy, precision, recall, f_score))
    print(classification_report(y, y_pred))
    return accuracy, precision, recall, f_score

In [40]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    prediction=[]
    true_prediction=[]
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:        
        
        text_ids = data['text_ids'].to(device, dtype = torch.long)
        text_mask = data['text_mask'].to(device, dtype = torch.long)
        targets = data['target'].to(device, dtype=torch.long)
        
        batch_size = text_ids.size(0)

        outputs = model(text_ids, text_mask)
        prediction.append(F.softmax(outputs).to('cpu').numpy())
        true_prediction.append(targets.to('cpu').numpy())
        # outputs = outputs.argmax(dim=1)
        
        loss = criterion(outputs, targets)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])   
    
    gc.collect()
    prediction = np.concatenate(prediction)
    true_prediction = np.concatenate(true_prediction)
    prediction=np.argmax(np.array(prediction),axis=1)
    print(print_statistics(np.array(true_prediction),prediction))
    print(f1_score(np.array(true_prediction),prediction, average='macro'))
    
    return f1_score(np.array(true_prediction),prediction, average='macro')

In [41]:
def run_training(model, optimizer, scheduler, device, num_epochs, fold):
    # To automatically log gradients
    
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_loss = 0
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_epoch_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CONFIG['device'], epoch=epoch)
        
        val_epoch_loss = valid_one_epoch(model, valid_loader, device=CONFIG['device'], 
                                         epoch=epoch)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        
       
        
        # deep copy the model
        if val_epoch_loss >= best_epoch_loss:
            print(f"Validation Loss Improved ({best_epoch_loss} ---> {val_epoch_loss})")
            best_epoch_loss = val_epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = f"/content/drive/MyDrive/wanlp/NADI/Saved_Models/Camel_bert/Loss-new-Fold-{fold}.bin"
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print("Model Saved")
            
        print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Loss: {:.4f}".format(best_epoch_loss))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history

In [42]:
def fetch_scheduler(optimizer):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CONFIG['T_max'], 
                                                   eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CONFIG['T_0'], 
                                                             eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == None:
        return None
        
    return scheduler

In [43]:
def prepare_loaders(train,valid):
    # df_train = train[train.kfold != fold].reset_index(drop=True)
    # df_valid = train[train.kfold == fold].reset_index(drop=True)
    # sampler = ImbalancedDatasetSampler(df_train)
    
    train_dataset = TrainDataset(train, tokenizer=tokenizer, max_length=CONFIG['max_length'])
    valid_dataset = TrainDataset(valid, tokenizer=tokenizer, max_length=CONFIG['max_length'])

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], 
                              num_workers=2, shuffle=True, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], 
                              num_workers=2, shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader

# Train

In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
train = pd.read_csv('/content/drive/MyDrive/wanlp/Datasets/NADI2022-Train/NADI2022-Train/Subtask1/NADI2022_Subtask1_TRAIN.tsv', sep='\t', lineterminator='\n')
valid = pd.read_csv('/content/drive/MyDrive/wanlp/Datasets/NADI2022-Train/NADI2022-Train/Subtask1/NADI2022_Subtask1_DEV.tsv', sep='\t', lineterminator='\n')

In [46]:
train.head()

,#1_id,#2_content,#3_label
0,TRAIN_15711,الطرحة في 61 النفر 10 جنيه,yemen
1,TRAIN_2257,كله ولا يطالبوا بارض فدك الله ياخدهم الله يعجل...,lebanon
2,TRAIN_5618,' لما اطلقتو تلك التغريدة وقتها كان في واحد سع...,algeria
3,TRAIN_7284,بجكولى بى خير,iraq
4,TRAIN_15841,يعم القمر براحه علينا,egypt


In [47]:
len(train)

20398

In [48]:
train['#3_label'].nunique()

18

In [49]:
valid['#3_label'].nunique()

18

In [50]:
# df_train=pd.concat([train,valid])

In [51]:
num_classes=train['#3_label'].nunique()

In [52]:
CONFIG = {"seed": 42,
          "epochs": 5,
          "train_batch_size": 16,
          "valid_batch_size": 64,
          "max_length": 512,
          "learning_rate": 5e-6,
          "scheduler": 'CosineAnnealingLR',
          "min_lr": 1e-8,
          "T_max": 500,
          "weight_decay": 1e-2,
          "n_fold": 5,
          "n_accumulate": 1,
          "num_classes": 18,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          }

In [53]:
tokenizer= AutoTokenizer.from_pretrained('CAMeL-Lab/bert-base-arabic-camelbert-da')


In [54]:
train.reset_index(inplace=True)

In [55]:
skf = StratifiedKFold(n_splits=CONFIG['n_fold'], shuffle=True, random_state=CONFIG['seed'])

for fold, ( _, val_) in enumerate(skf.split(X=train, y=train['#3_label'])):
    train.loc[val_ , "kfold"] = int(fold)
    
train["kfold"] = train["kfold"].astype(int)

In [56]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train['#3_label'] = le.fit_transform(train['#3_label'].values)

In [57]:
valid['#3_label'] = le.transform(valid['#3_label'].values)

In [58]:

    
# Create Dataloaders
train_loader, valid_loader = prepare_loaders(train,valid)

model = NADIModel(pretrained_path='CAMeL-Lab/bert-base-arabic-camelbert-mix')
model.to(CONFIG['device'])
torch.cuda.empty_cache()
# model.load_state_dict(torch.load('/content/drive/MyDrive/wanlp/NADI/Saved_Models/Camel_bert/Loss-Fold-4.bin'))

# Define Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
scheduler = fetch_scheduler(optimizer)

model, history = run_training(model, optimizer, scheduler,
                              device=CONFIG['device'],
                              num_epochs=CONFIG['epochs'],
                              fold=1)


del model, history, train_loader, valid_loader
_ = gc.collect()
print()

Some weights of the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[INFO] Using GPU: Tesla T4



100%|██████████| 77/77 [02:43<00:00,  2.12s/it, Epoch=1, LR=2.13e-6, Valid_Loss=1.98]


Accuracy: 0.421
Precision: 0.370
Recall: 0.421
F_score: 0.342

              precision    recall  f1-score   support

           0       0.42      0.50      0.46       430
           1       0.00      0.00      0.00        52
           2       0.66      0.88      0.75      1041
           3       0.40      0.59      0.48       664
           4       0.00      0.00      0.00       104
           5       0.27      0.81      0.41       520
           6       0.00      0.00      0.00       105
           7       0.19      0.10      0.13       157
           8       0.28      0.07      0.11       314
           9       0.75      0.01      0.03       207
          10       0.17      0.03      0.05       355
          11       0.00      0.00      0.00       104
          12       0.00      0.00      0.00        52
          13       0.00      0.00      0.00        53
          14       0.19      0.13      0.16       278
          15       0.50      0.08      0.13       173
          16      

100%|██████████| 77/77 [02:43<00:00,  2.12s/it, Epoch=2, LR=1.23e-7, Valid_Loss=1.94]


...

Accuracy: 0.443
Precision: 0.410
Recall: 0.443
F_score: 0.390

              precision    recall  f1-score   support

           0       0.41      0.51      0.46       430
           1       0.00      0.00      0.00        52
           2       0.66      0.88      0.75      1041
           3       0.44      0.59      0.50       664
           4       0.00      0.00      0.00       104
           5       0.34      0.63      0.44       520
           6       0.00      0.00      0.00       105
           7       0.25      0.12      0.16       157
           8       0.33      0.20      0.25       314
           9       0.78      0.09      0.16       207
          10       0.23      0.26      0.24       355
          11       0.17      0.01      0.02       104
          12       0.00      0.00      0.00        52
          13       0.37      0.25      0.30        53
          14       0.25      0.15      0.19       278
          15       0.25      0.06      0.09       173
          16      

100%|██████████| 77/77 [02:42<00:00,  2.12s/it, Epoch=4, LR=4.56e-6, Valid_Loss=1.93]


Accuracy: 0.448
Precision: 0.437
Recall: 0.448
F_score: 0.400

              precision    recall  f1-score   support

           0       0.39      0.56      0.46       430
           1       0.00      0.00      0.00        52
           2       0.64      0.90      0.74      1041
           3       0.62      0.49      0.55       664
           4       0.00      0.00      0.00       104
           5       0.30      0.71      0.42       520
           6       0.10      0.01      0.02       105
           7       0.35      0.13      0.19       157
           8       0.37      0.18      0.24       314
           9       0.81      0.08      0.15       207
          10       0.25      0.26      0.26       355
          11       0.25      0.02      0.04       104
          12       0.00      0.00      0.00        52
          13       0.35      0.25      0.29        53
          14       0.26      0.17      0.21       278
          15       0.31      0.11      0.16       173
          16      

100%|██████████| 77/77 [02:40<00:00,  2.08s/it, Epoch=5, LR=7.97e-7, Valid_Loss=1.9]


Accuracy: 0.456
Precision: 0.426
Recall: 0.456
F_score: 0.412

              precision    recall  f1-score   support

           0       0.47      0.47      0.47       430
           1       0.00      0.00      0.00        52
           2       0.64      0.89      0.75      1041
           3       0.50      0.59      0.54       664
           4       0.00      0.00      0.00       104
           5       0.34      0.61      0.43       520
           6       0.20      0.01      0.02       105
           7       0.41      0.12      0.19       157
           8       0.36      0.32      0.34       314
           9       0.67      0.09      0.15       207
          10       0.27      0.27      0.27       355
          11       0.11      0.04      0.06       104
          12       0.00      0.00      0.00        52
          13       0.33      0.25      0.28        53
          14       0.24      0.21      0.22       278
          15       0.34      0.14      0.20       173
          16      